<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Plan" data-toc-modified-id="Plan-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Plan</a></span></li><li><span><a href="#Load-&amp;-prepare-data" data-toc-modified-id="Load-&amp;-prepare-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load &amp; prepare data</a></span></li><li><span><a href="#Create-graph,-populate-with-nodes-&amp;-edges" data-toc-modified-id="Create-graph,-populate-with-nodes-&amp;-edges-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Create graph, populate with nodes &amp; edges</a></span></li><li><span><a href="#Graph-created-&amp;-geoff'd-for-easy-imports" data-toc-modified-id="Graph-created-&amp;-geoff'd-for-easy-imports-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Graph created &amp; geoff'd for easy imports</a></span></li><li><span><a href="#Next-steps:" data-toc-modified-id="Next-steps:-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Next steps:</a></span></li><li><span><a href="#hosted-it-in-neo4j.-let's-explore-some-queries" data-toc-modified-id="hosted-it-in-neo4j.-let's-explore-some-queries-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>hosted it in neo4j. let's explore some queries</a></span></li></ul></div>

In [1]:
import networkx as nx
import pandas as pd
import selenium
import random
import time
import os
import io
import json 
import pickle
import requests
import xmltodict
import pprint
import json
import config
import matplotlib.pyplot as plt
import seaborn as sns
%matploblib inline

import neo4j
import neonx # fancy

import xml.etree.ElementTree as ET
from io import StringIO
from selenium import webdriver 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException

UsageError: Line magic function `%matploblib` not found.


In [ ]:
def pickle_obj(obj, filename): # designed with list of strings in mind
    urls = open(filename,"w+")
    with open(filename, 'wb') as f:
        pickle.dump(obj, f)
def unpickle_obj(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

def add_industry_nodes(G, industries): # pass graph, list of industries
    for industry in industries:
        G.add_node(str(industry), size=0)
        print(type(industry), type(G.node[industry]))
    return G # returns modified graph

# Plan
- load congressmember industry data
    - create graph using networkx
        - create nodes, edges in graph using data
            - convert to geoff string for use in Gephi

# Load & prepare data

In [2]:
def pickle_obj(obj, filename): # designed with list of strings in mind
    urls = open(filename,"w+")
    with open(filename, 'wb') as f:
        pickle.dump(obj, f)
def unpickle_obj(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

In [3]:
with open('first_200', 'rb') as f: # i should probably just know the syntax
    mems = pickle.load(f) # is the time saving method worth the lack of knowledge? the eternal question

In [4]:
mems[0][1] # list of dictionaries

{'name': 'Alma Adams (D)',
 'party': 'D',
 'id': 'N00035451',
 'Public Sector Unions': {'code': 'P04',
  'name': 'Public Sector Unions',
  'indiv': '0',
  'pac': '44500',
  'total': '44500'},
 'Industrial Unions': {'code': 'P02',
  'name': 'Industrial Unions',
  'indiv': '0',
  'pac': '41000',
  'total': '41000'},
 'Building Trade Unions': {'code': 'P01',
  'name': 'Building Trade Unions',
  'indiv': '0',
  'pac': '32500',
  'total': '32500'},
 'Health Professionals': {'code': 'H01',
  'name': 'Health Professionals',
  'indiv': '11880',
  'pac': '18000',
  'total': '29880'},
 'Insurance': {'code': 'F09',
  'name': 'Insurance',
  'indiv': '0',
  'pac': '23000',
  'total': '23000'},
 'Securities & Investment': {'code': 'F07',
  'name': 'Securities & Investment',
  'indiv': '0',
  'pac': '22500',
  'total': '22500'},
 'Leadership PACs': {'code': 'Q03',
  'name': 'Leadership PACs',
  'indiv': '0',
  'pac': '19000',
  'total': '19000'},
 'Lawyers/Law Firms': {'code': 'K01',
  'name': 'Lawye

# Create graph, populate with nodes & edges

In [5]:
def get_industries(mems): # pass list-list-dict-dict data
    industries = [] # returns list of unique industries
    for chamber in mems: # this is awfully nested
        for member in chamber: # dict
            for k, v in member.items():
                if k!='party' and k!='name' and k!='id':
                    if k not in industries:
                        industries.append(k)
    return industries

def add_industry_nodes(G, industries): # pass graph, list of industries
    for industry in industries:
        G.add_node(str(industry), size=0, ind=industry)
        print(type(industry), type(G.node[industry]))
    return G # returns modified graph

In [6]:
g = nx.Graph()
industries = get_industries(mems)
g = add_industry_nodes(g, industries)
g.nodes()

<class 'str'> <class 'dict'>
<class 'str'> <class 'dict'>
<class 'str'> <class 'dict'>
<class 'str'> <class 'dict'>
<class 'str'> <class 'dict'>
<class 'str'> <class 'dict'>
<class 'str'> <class 'dict'>
<class 'str'> <class 'dict'>
<class 'str'> <class 'dict'>
<class 'str'> <class 'dict'>
<class 'str'> <class 'dict'>
<class 'str'> <class 'dict'>
<class 'str'> <class 'dict'>
<class 'str'> <class 'dict'>
<class 'str'> <class 'dict'>
<class 'str'> <class 'dict'>
<class 'str'> <class 'dict'>
<class 'str'> <class 'dict'>
<class 'str'> <class 'dict'>
<class 'str'> <class 'dict'>
<class 'str'> <class 'dict'>
<class 'str'> <class 'dict'>
<class 'str'> <class 'dict'>
<class 'str'> <class 'dict'>
<class 'str'> <class 'dict'>
<class 'str'> <class 'dict'>
<class 'str'> <class 'dict'>
<class 'str'> <class 'dict'>
<class 'str'> <class 'dict'>
<class 'str'> <class 'dict'>
<class 'str'> <class 'dict'>
<class 'str'> <class 'dict'>
<class 'str'> <class 'dict'>
<class 'str'> <class 'dict'>
<class 'str'> 

NodeView(('Health Professionals', 'Crop Production & Basic Processing', 'Agricultural Services/Products', 'General Contractors', 'Air Transport', 'Leadership PACs', 'Defense Aerospace', 'Health Services/HMOs', 'Oil & Gas', 'Hospitals/Nursing Homes', 'Public Sector Unions', 'Industrial Unions', 'Building Trade Unions', 'Insurance', 'Securities & Investment', 'Lawyers/Law Firms', 'Electric Utilities', 'Misc Defense', 'Misc Manufacturing & Distributing', 'Food Processing & Sales', 'Defense Electronics', 'Lobbyists', 'Commercial Banks', 'Retired', 'Real Estate', 'Education', 'Pharmaceuticals/Health Products', 'Forestry & Forest Products', 'Republican/Conservative', 'Retail Sales', 'Non-Profit Institutions', 'Casinos/Gambling', 'Mining', 'Livestock', 'Automotive', 'Chemical & Related Manufacturing', 'Construction Services', 'Candidate Committees', 'Building Materials & Equipment', 'Transportation Unions', 'Democratic/Liberal', 'TV/Movies/Music', "Women's Issues", 'Misc Energy', 'Telecom Ser

In [7]:
help(nx.Graph.add_node)

Help on function add_node in module networkx.classes.graph:

add_node(self, node_for_adding, **attr)
    Add a single node `node_for_adding` and update node attributes.
    
    Parameters
    ----------
    node_for_adding : node
        A node can be any hashable Python object except None.
    attr : keyword arguments, optional
        Set or change node attributes using key=value.
    
    See Also
    --------
    add_nodes_from
    
    Examples
    --------
    >>> G = nx.Graph()  # or DiGraph, MultiGraph, MultiDiGraph, etc
    >>> G.add_node(1)
    >>> G.add_node('Hello')
    >>> K3 = nx.Graph([(0, 1), (1, 2), (2, 0)])
    >>> G.add_node(K3)
    >>> G.number_of_nodes()
    3
    
    Use keywords set/change node attributes:
    
    >>> G.add_node(1, size=10)
    >>> G.add_node(3, weight=0.4, UTM=('13S', 382871, 3972649))
    
    Notes
    -----
    A hashable object is one that can be used as a key in a Python
    dictionary. This includes strings, numbers, tuples of strings
 

In [8]:
def populate_graph(G, mems, industries): # add member nodes + edges
    edges = []
    ec=0
    for chamber in mems:
        for c, member in enumerate(chamber):
            if len(member)!=0:
                try:
                    mem_node = G.add_node(member['name'], size=0)
                    total_mem_size = 0
                    print(c)
                    for k, v in member.items():
                        if k!='party' and k!='name' and k!='id': # industry dict
                            G.add_edge(member['name'], v['name'], weight=v['total'])
#                             G[member['name']][v['name']]['party'] = member['party']
                            total_mem_size += int(v['total']) # update member's total bribery $
                            G.node[v['name']]['size'] += int(v['total']) # update industry's total bribery $
                    G.node[member['name']]['size'] = total_mem_size # reassign mem_node size
                    G.node[member['name']]['party'] = member['party']
                    G.node[member['name']]['name'] = member['name'] # give a name property
                except:
                    ec+=1 # count errors to ensure data integrity
    print(f'{ec} errors')
    return G

In [9]:
g = populate_graph(g, mems, industries)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
0 errors


In [11]:
# graph.nodes

In [14]:
pickle_obj(g, '200_graph')

In [21]:
g.edges()

EdgeView([('Health Professionals', 'Ralph Abraham (R)'), ('Health Professionals', 'Alma Adams (D)'), ('Health Professionals', 'Pete Aguilar (D)'), ('Health Professionals', 'Richard W Allen (R)'), ('Health Professionals', 'Justin Amash (R)'), ('Health Professionals', 'Mark Amodei (R)'), ('Health Professionals', 'Jodey Arrington (R)'), ('Health Professionals', 'Brian Babin (R)'), ('Health Professionals', 'Troy Balderson (R)'), ('Health Professionals', 'Jim Banks (R)'), ('Health Professionals', 'Lou Barletta (R)'), ('Health Professionals', 'Andy Barr (R)'), ('Health Professionals', 'Joe Barton (R)'), ('Health Professionals', 'Karen Bass (D)'), ('Health Professionals', 'Ami Bera (D)'), ('Health Professionals', 'John Bergman (R)'), ('Health Professionals', 'Jaime Herrera Beutler (R)'), ('Health Professionals', 'Don Beyer (D)'), ('Health Professionals', 'Andy Biggs (R)'), ('Health Professionals', 'Gus Bilirakis (R)'), ('Health Professionals', 'Mike Bishop (R)'), ('Health Professionals', 'San

In [15]:
graph = g

In [16]:
import neonx

In [17]:
# create a Networkx graph
# LINKS_TO is the relatioship name between the nodes
data = neonx.get_geoff(graph, "bribe")

In [18]:
# write graph to Neo4J db hosted locally
results = neonx.write_to_neo("http://localhost:7474/db/data/", graph, 'bribe')

In [75]:
help(neonx.write_to_neo)

Help on function write_to_neo in module neonx.neo:

write_to_neo(server_url, graph, edge_rel_name, encoder=None)
    Write the `graph` as Geoff string. The edges between the nodes
    have relationship name `edge_rel_name`. The code
    below shows a simple example::
    
        from neonx import write_to_neo
    
        # create a graph
        import networkx as nx
        G = nx.Graph()
        G.add_nodes_from([1, 2, 3])
        G.add_edge(1, 2)
        G.add_edge(2, 3)
    
    
        # save graph to neo4j
        results = write_to_neo("http://localhost:7474/db/data/", G, 'LINKS_TO')
    
    If the properties are not json encodable, please pass a custom JSON encoder
    class. See `JSONEncoder
    <http://docs.python.org/2/library/json.html#json.JSONEncoder/>`_.
    
    :param server_url: Server URL for the Neo4j server.
    :param graph: A NetworkX Graph or a DiGraph
    :param edge_rel_name: Relationship name between the nodes
    :param optional encoder: JSONEncoder obje

In [19]:
pickle_obj(data, '200')

In [32]:
pwd

'/Users/mark/Documents/flatiron/projects/congress_bribes/graphing_congressional_lobbying/graphing_congress_lobbying_industries'

In [26]:
nx.write_gexf(graph, "/Users/mark/Documents/flatiron/projects/congress_bribes/graphing_congressional_lobbying/graphing_congress_lobbying_industries/200_geoff.gexf")

In [135]:
with open('graph_geoff', 'w') as f:
    nx.write_gexf(graph, "400_geoff_new_n.gexf")

# Graph created & geoff'd for easy imports
- I've got it running in Gephi right now. it's quite colourful.

# hosted it in neo4j. let's explore some queries

In [55]:
g['Brian Babin (R)']

AtlasView({'Health Professionals': {'weight': '172491'}, 'Oil & Gas': {'weight': '80807'}, 'Air Transport': {'weight': '65500'}, 'Retired': {'weight': '59656'}, 'Chemical & Related Manufacturing': {'weight': '54000'}, 'Construction Services': {'weight': '35700'}, 'Real Estate': {'weight': '34274'}, 'Automotive': {'weight': '33600'}, 'Lawyers/Law Firms': {'weight': '33249'}, 'General Contractors': {'weight': '29681'}})